# Using the xfDNN Quantizer to quantize caffe models

## Introduction 

In this part of the lab, we will look at quantizing 32-bit floating point models to Int16 or Int8 inpreparation for deployment. Deploying Int16/8 models dramatically improves inference deployment and lowers latency. While floating point precision is useful in model training, it is more energy efficient as well as lower latency to deploy models in lower precison. 

The xfDNN Quantizer performs a technique of quantization known as recalibration. This technique does not require full retraining of the model, and can be accomplished in a matter of seconds, as you will see below. It also allows you to maintain the accuracy of the high precision model.

Quantization of the model does not alter the orginal high precision model, rather, it calculates the dynamic range of the model and produces scaling parameters recorded in a json file, which will be used by the xDNN overlay during execution of the network/model. Quantization of the model is an offline process that only needs to be performed once per model. The quantizer produces an optimal target quantization from a given network (prototxt and caffemodel) and calibration set (unlabeled input images) without requiring hours of retraining or a labeled dataset.

In this lab, we will look at quantizing an optimized model generated from Part 1, defined in Caffe prototxt and caffemodel, to Int16 and Int8.  Depending on your earlier notebook this will be either a GoogLeNet-v1 or Resnet-50 model.

First we will run through an example, then you will get a chance to try the quantizer yourself. 

### 1. Import required packages 

In [1]:
import os,sys
from __future__ import print_function

# Bring in Xilinx ML-Suite Compiler
from xfdnn.tools.quantize.quantize import CaffeFrontend as xfdnnQuantizer

### 1. Create Quantizer Instance and run it

To simplify handling of arguments, a config dictionary is used. Take a look at the dictionary below.

The arguments that need to be passed are:
- `outmodel` - Filename generated by the compiler for the optimized prototxt and caffemodel.
- `quantizecfg` - Output JSON filename of quantization scaling parameters. 
- `bitwidths` - Desired precision from quantizer. This is to set the precision for [image data, weight bitwidth, conv output]. All three values need to be set to the same setting. The valid options are `16` for Int16 and `8` for Int8.  
- `in_shape` - Sets the desired input image size of the first layer. Images will be resized to these demensions and must match the network data/placeholder layer.
- `transpose` - Images start as H,W,C (H=0,W=1,C=2) transpose swaps to C,H,W (2,0,1) for typical networks.
- `channel_swap` - Depending on network training and image read, can swap from RGB (R=0,G=1,B=2) to BGR (2,1,0).
- `raw_scale` - Depending on network training, scale pixel values before mean subtraction.
- `img_mean` - Depending on network training, subtract image mean if available.
- `input_scale` - Depending on network training, scale after subtracting mean.
- `calibration_size` - Number of images the quantizer will use to calculate the dynamic range. 
- `calibration_directory` - Location of dir of images used for the calibration process. 

Below is an example with all the parameters filled in. `channel_swap` `raw_scale` `img_mean` `input_scale` are expert parameters that should be left in the default positions, indicated below. 

In [2]:
# Use a config dictionary to pass parameters to the compiler
config = {}

config["caffemodel"] = "work/optimized_model" # String for naming intermediate prototxt, caffemodel

# Quantizer Arguments
#config["outmodel"] = Defined in Step 1 # String for naming intermediate prototxt, caffemodel
config["quantizecfg"] = "work/quantization_params.json" # Quantizer will generate quantization params
config["bitwidths"] = [16,16,16] # Supported quantization precision
config["in_shape"] = [3,224,224] # Images will be resized to this shape -> Needs to match prototxt
config["transpose"] = [2,0,1] # (H,W,C)->(C,H,W) transpose argument to quantizer
config["channel_swap"] = [2,1,0] # (R,G,B)->(B,G,R) Channel Swap argument to quantizer
config["raw_scale"] = 255.0 # Input multiplier, Images are scaled by this factor before mean subtraction
config["img_mean"] = [104.007, 116.669, 122.679] # Mean of the training set used for mean subtraction
config["input_scale"] = 1.0 # Input multiplier, Images are scaled by this factor after mean subtraction
config["calibration_size"] = 8 # Number of calibration images quantizer will use
config["calibration_directory"] = "../xfdnn/tools/quantize/calibration_directory" # Directory of images

quantizer = xfdnnQuantizer(
    deploy_model=config["caffemodel"]+".prototxt",        # Model filename: input file
    weights=config["caffemodel"]+".caffemodel",           # Floating Point weights
    output_json=config["quantizecfg"],                    # Quantization JSON output filename
    bitwidths=config["bitwidths"],                        # Fixed Point precision: 8,8,8 or 16,16,16
    dims=config["in_shape"],                              # Image dimensions [C,H,W]
    transpose=config["transpose"],                        # Transpose argument to caffe transformer
    channel_swap=config["channel_swap"],                  # Channel swap argument to caffe transfomer
    raw_scale=config["raw_scale"],                        # Raw scale argument to caffe transformer
    mean_value=config["img_mean"],                        # Image mean per channel to caffe transformer
    input_scale=config["input_scale"],                    # Input scale argument to caffe transformer
    calibration_size=config["calibration_size"],          # Number of calibration images to use
    calibration_directory=config["calibration_directory"] # Directory containing calbration images
)

# Invoke quantizer
try:
    quantizer.quantize()

    import json
    data = json.loads(open(config["quantizecfg"]).read())
    print("**********\nSuccessfully produced quantization JSON file for %d layers.\n"%len(data['network']))
except Exception as e:
    print("Failed to quantize:",e)

Mean : [104.007 116.669 122.679]
Adding ../xfdnn/tools/quantize/calibration_directory/13923040300_b4c8521b4d_z.jpg to calibration batch.
Adding ../xfdnn/tools/quantize/calibration_directory/14931486720_37bd588ce9_z.jpg to calibration batch.
Adding ../xfdnn/tools/quantize/calibration_directory/15439525724_97d7cc2c81_z.jpg to calibration batch.
Adding ../xfdnn/tools/quantize/calibration_directory/3591612840_33710806df_z.jpg to calibration batch.
Adding ../xfdnn/tools/quantize/calibration_directory/4814953542_de4b973dc2_z.jpg to calibration batch.


/home/bryanloz/anaconda2/envs/MLsuite/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Adding ../xfdnn/tools/quantize/calibration_directory/7291910830_86a8ebb15d_z.jpg to calibration batch.
Adding ../xfdnn/tools/quantize/calibration_directory/7647574936_ffebfa2bea_z.jpg to calibration batch.
Adding ../xfdnn/tools/quantize/calibration_directory/8289365270_82b20ef781_z.jpg to calibration batch.
--------------------------------------------------------------------------------
Processing layer 0 of 139
Layer Name:data Type:Input
Inputs: [], Outputs: ['data']
Quantizing layer output...
Min:  0 , Max:  150.993
n:  32768 , len(bin_edges):  1099
Mean : th_layer_out:  150.9929962158203 , sf_layer_out:  0.00460808118582172
bw_layer_out:  16
th_layer_out:  150.9929962158203
--------------------------------------------------------------------------------
Processing layer 1 of 139
Layer Name:conv1 Type:Convolution
Inputs: ['data'], Outputs: ['conv1']
Quantizing conv input layer ... conv1
Threshold in shape= ()
Quantizing conv weights for layer conv1...
Threshold params shape= (64,)
Mi

bw_layer_out:  16
th_layer_out:  10.822406768798828
--------------------------------------------------------------------------------
Processing layer 16 of 139
Layer Name:res2b_branch2b Type:Convolution
Inputs: ['res2b_branch2a'], Outputs: ['res2b_branch2b']
Quantizing conv input layer ... res2b_branch2b
Threshold in shape= ()
Quantizing conv weights for layer res2b_branch2b...
Threshold params shape= (64,)
Min:  0 , Max:  11.766703
n:  32768 , len(bin_edges):  1269
Mean : th_layer_out:  11.766702651977539 , sf_layer_out:  0.0003591022263856178
Threshold out shape= ()
Min:  0 , Max:  11.766703
n:  32768 , len(bin_edges):  1269
Mean : th_layer_out:  11.766702651977539 , sf_layer_out:  0.0003591022263856178
bw_layer_in:  16
th_layer_in:  10.822406768798828
bw_layer_out:  16
th_layer_out:  11.766702651977539
--------------------------------------------------------------------------------
Processing layer 17 of 139
Layer Name:res2b_branch2b_relu Type:ReLU
Inputs: ['res2b_branch2b'], Output

n:  32768 , len(bin_edges):  898
Mean : th_layer_out:  8.970563888549805 , sf_layer_out:  0.00027376823903774546
Threshold out shape= ()
Min:  0 , Max:  8.970564
n:  32768 , len(bin_edges):  898
Mean : th_layer_out:  8.970563888549805 , sf_layer_out:  0.00027376823903774546
bw_layer_in:  16
th_layer_in:  8.936508178710938
bw_layer_out:  16
th_layer_out:  8.970563888549805
--------------------------------------------------------------------------------
Processing layer 34 of 139
Layer Name:res3a_branch2b_relu Type:ReLU
Inputs: ['res3a_branch2b'], Outputs: ['res3a_branch2b']
Min:  0 , Max:  8.553736
n:  32768 , len(bin_edges):  897
Mean : th_layer_out:  8.553735733032227 , sf_layer_out:  0.0002610472650237198
bw_layer_out:  16
th_layer_out:  8.553735733032227
--------------------------------------------------------------------------------
Processing layer 35 of 139
Layer Name:res3a_branch2c Type:Convolution
Inputs: ['res3a_branch2b'], Outputs: ['res3a_branch2c']
Quantizing conv input lay

...

n:  32768 , len(bin_edges):  635
Mean : th_layer_out:  9.488764762878418 , sf_layer_out:  0.00028958295733141323
bw_layer_out:  16
th_layer_out:  9.488764762878418
--------------------------------------------------------------------------------
Processing layer 66 of 139
Layer Name:res4a_branch2b Type:Convolution
Inputs: ['res4a_branch2a'], Outputs: ['res4a_branch2b']
Quantizing conv input layer ... res4a_branch2b
Threshold in shape= ()
Quantizing conv weights for layer res4a_branch2b...
Threshold params shape= (256,)
Min:  0 , Max:  8.69502
n:  32768 , len(bin_edges):  635
Mean : th_layer_out:  8.695019721984863 , sf_layer_out:  0.00026535904177937754
Threshold out shape= ()
Min:  0 , Max:  8.69502
n:  32768 , len(bin_edges):  635
Mean : th_layer_out:  8.695019721984863 , sf_layer_out:  0.00026535904177937754
bw_layer_in:  16
th_layer_in:  9.488764762878418
bw_layer_out:  16
th_layer_out:  8.695019721984863
------------------------------------------------------------------------------

n:  32768 , len(bin_edges):  635
Mean : th_layer_out:  7.503726959228516 , sf_layer_out:  0.00022900256231051105
bw_layer_out:  16
th_layer_out:  7.503726959228516
--------------------------------------------------------------------------------
Processing layer 82 of 139
Layer Name:res4c_branch2b Type:Convolution
Inputs: ['res4c_branch2a'], Outputs: ['res4c_branch2b']
Quantizing conv input layer ... res4c_branch2b
Threshold in shape= ()
Quantizing conv weights for layer res4c_branch2b...
Threshold params shape= (256,)
Min:  0 , Max:  8.24147
n:  32768 , len(bin_edges):  635
Mean : th_layer_out:  8.241470336914062 , sf_layer_out:  0.0002515173905732616
Threshold out shape= ()
Min:  0 , Max:  8.24147
n:  32768 , len(bin_edges):  635
Mean : th_layer_out:  8.241470336914062 , sf_layer_out:  0.0002515173905732616
bw_layer_in:  16
th_layer_in:  7.503726959228516
bw_layer_out:  16
th_layer_out:  8.241470336914062
--------------------------------------------------------------------------------

bw_layer_out:  16
th_layer_out:  10.931528091430664
--------------------------------------------------------------------------------
Processing layer 98 of 139
Layer Name:res4e_branch2b Type:Convolution
Inputs: ['res4e_branch2a'], Outputs: ['res4e_branch2b']
Quantizing conv input layer ... res4e_branch2b
Threshold in shape= ()
Quantizing conv weights for layer res4e_branch2b...
Threshold params shape= (256,)
Min:  0 , Max:  13.177458
n:  32768 , len(bin_edges):  635
Mean : th_layer_out:  13.177457809448242 , sf_layer_out:  0.0004021563710271994
Threshold out shape= ()
Min:  0 , Max:  13.177458
n:  32768 , len(bin_edges):  635
Mean : th_layer_out:  13.177457809448242 , sf_layer_out:  0.0004021563710271994
bw_layer_in:  16
th_layer_in:  10.931528091430664
bw_layer_out:  16
th_layer_out:  13.177457809448242
--------------------------------------------------------------------------------
Processing layer 99 of 139
Layer Name:res4e_branch2b_relu Type:ReLU
Inputs: ['res4e_branch2b'], Outputs

Min:  0 , Max:  7.7102456
n:  32768 , len(bin_edges):  450
Mean : th_layer_out:  7.710245609283447 , sf_layer_out:  0.00023530520368918262
Threshold out shape= ()
Min:  0 , Max:  7.7102456
n:  32768 , len(bin_edges):  450
Mean : th_layer_out:  7.710245609283447 , sf_layer_out:  0.00023530520368918262
bw_layer_in:  16
th_layer_in:  21.89035987854004
bw_layer_out:  16
th_layer_out:  7.710245609283447
--------------------------------------------------------------------------------
Processing layer 114 of 139
Layer Name:res5a_branch2a_relu Type:ReLU
Inputs: ['res5a_branch2a'], Outputs: ['res5a_branch2a']
Min:  0 , Max:  7.7006645
n:  32768 , len(bin_edges):  449
Mean : th_layer_out:  7.700664520263672 , sf_layer_out:  0.00023501280313314223
bw_layer_out:  16
th_layer_out:  7.700664520263672
--------------------------------------------------------------------------------
Processing layer 115 of 139
Layer Name:res5a_branch2b Type:Convolution
Inputs: ['res5a_branch2a'], Outputs: ['res5a_branc

Threshold params shape= (512,)
Min:  0 , Max:  9.011212
n:  32768 , len(bin_edges):  450
Mean : th_layer_out:  9.011212348937988 , sf_layer_out:  0.00027500876946128693
Threshold out shape= ()
Min:  0 , Max:  9.011212
n:  32768 , len(bin_edges):  450
Mean : th_layer_out:  9.011212348937988 , sf_layer_out:  0.00027500876946128693
bw_layer_in:  16
th_layer_in:  46.5992317199707
bw_layer_out:  16
th_layer_out:  9.011212348937988
--------------------------------------------------------------------------------
Processing layer 130 of 139
Layer Name:res5c_branch2a_relu Type:ReLU
Inputs: ['res5c_branch2a'], Outputs: ['res5c_branch2a']
Min:  0 , Max:  9.011212
n:  32768 , len(bin_edges):  449
Mean : th_layer_out:  9.011212348937988 , sf_layer_out:  0.00027500876946128693
bw_layer_out:  16
th_layer_out:  9.011212348937988
--------------------------------------------------------------------------------
Processing layer 131 of 139
Layer Name:res5c_branch2b Type:Convolution
Inputs: ['res5c_branch2

### 2. Try it yourself by changing the quantization precision

Now that you have had a chance to see how this works, it's time to get some hands on experience.  
Change the following from the example above:
1. Precision of quantization by adjusting `bitwidth`

Below, replace `value` with one of the supported precision types. [8,8,8] or [16,16,16]

In [3]:
# Since we already have an instance of the quantizer, you can just update these params:

quantizer.bitwidths = [8,8,8]

# Invoke quantizer
try:
    quantizer.quantize()

    import json
    data = json.loads(open(config["quantizecfg"]).read())
    print("**********\nSuccessfully produced quantization JSON file for %d layers.\n"%len(data['network']))
except Exception as e:
    print("Failed to quantize:",e)

Mean : [104.007 116.669 122.679]
Adding ../xfdnn/tools/quantize/calibration_directory/13923040300_b4c8521b4d_z.jpg to calibration batch.
Adding ../xfdnn/tools/quantize/calibration_directory/14931486720_37bd588ce9_z.jpg to calibration batch.
Adding ../xfdnn/tools/quantize/calibration_directory/16247716843_b419e8b111_z.jpg to calibration batch.
Adding ../xfdnn/tools/quantize/calibration_directory/3591612840_33710806df_z.jpg to calibration batch.
Adding ../xfdnn/tools/quantize/calibration_directory/4788821373_441cd29c9f_z.jpg to calibration batch.
Adding ../xfdnn/tools/quantize/calibration_directory/7291910830_86a8ebb15d_z.jpg to calibration batch.
Adding ../xfdnn/tools/quantize/calibration_directory/7647574936_ffebfa2bea_z.jpg to calibration batch.
Adding ../xfdnn/tools/quantize/calibration_directory/8289365270_82b20ef781_z.jpg to calibration batch.
--------------------------------------------------------------------------------
Processing layer 0 of 139
Layer Name:data Type:Input
Inputs

Mean : th_layer_out:  8.183233490501669 , sf_layer_out:  0.06443490937402889
bw_layer_in:  8
th_layer_in:  4.309513570759188
bw_layer_out:  8
th_layer_out:  8.183233490501669
--------------------------------------------------------------------------------
Processing layer 15 of 139
Layer Name:res2b_branch2a_relu Type:ReLU
Inputs: ['res2b_branch2a'], Outputs: ['res2b_branch2a']
Min:  0 , Max:  6.849128
n:  128 , len(bin_edges):  1269
Mean : th_layer_out:  5.733713822785988 , sf_layer_out:  0.04514735293532274
bw_layer_out:  8
th_layer_out:  5.733713822785988
--------------------------------------------------------------------------------
Processing layer 16 of 139
Layer Name:res2b_branch2b Type:Convolution
Inputs: ['res2b_branch2a'], Outputs: ['res2b_branch2b']
Quantizing conv input layer ... res2b_branch2b
Threshold in shape= ()
Quantizing conv weights for layer res2b_branch2b...
Threshold params shape= (64,)
Min:  0 , Max:  10.545771
n:  128 , len(bin_edges):  1269
Mean : th_layer_out

Mean : th_layer_out:  6.174500831393494 , sf_layer_out:  0.04861811678262593
Threshold out shape= ()
Min:  0 , Max:  7.0063596
n:  128 , len(bin_edges):  898
Mean : th_layer_out:  6.174500831393494 , sf_layer_out:  0.04861811678262593
bw_layer_in:  8
th_layer_in:  4.929535718124059
bw_layer_out:  8
th_layer_out:  6.174500831393494
--------------------------------------------------------------------------------
Processing layer 32 of 139
Layer Name:res3a_branch2a_relu Type:ReLU
Inputs: ['res3a_branch2a'], Outputs: ['res3a_branch2a']
Min:  0 , Max:  6.990563
n:  128 , len(bin_edges):  897
Mean : th_layer_out:  5.738347125638809 , sf_layer_out:  0.04518383563495125
bw_layer_out:  8
th_layer_out:  5.738347125638809
--------------------------------------------------------------------------------
Processing layer 33 of 139
Layer Name:res3a_branch2b Type:Convolution
Inputs: ['res3a_branch2a'], Outputs: ['res3a_branch2b']
Quantizing conv input layer ... res3a_branch2b
Threshold in shape= ()
Qu

n:  128 , len(bin_edges):  897
Mean : th_layer_out:  3.9836774026708945 , sf_layer_out:  0.031367538603707834
bw_layer_out:  8
th_layer_out:  3.9836774026708945
--------------------------------------------------------------------------------
Processing layer 49 of 139
Layer Name:res3c_branch2b Type:Convolution
Inputs: ['res3c_branch2a'], Outputs: ['res3c_branch2b']
Quantizing conv input layer ... res3c_branch2b
Threshold in shape= ()
Quantizing conv weights for layer res3c_branch2b...
Threshold params shape= (128,)
Min:  0 , Max:  9.915917
n:  128 , len(bin_edges):  898
Mean : th_layer_out:  7.190974656023176 , sf_layer_out:  0.05662184768522186
Threshold out shape= ()
Min:  0 , Max:  9.915917
n:  128 , len(bin_edges):  898
Mean : th_layer_out:  7.190974656023176 , sf_layer_out:  0.05662184768522186
bw_layer_in:  8
th_layer_in:  3.9836774026708945
bw_layer_out:  8
th_layer_out:  7.190974656023176
--------------------------------------------------------------------------------
Processin

Mean : th_layer_out:  5.0812517987440815 , sf_layer_out:  0.040009856683024266
Threshold out shape= ()
Min:  0 , Max:  8.695044
n:  128 , len(bin_edges):  635
Mean : th_layer_out:  5.0812517987440815 , sf_layer_out:  0.040009856683024266
bw_layer_in:  8
th_layer_in:  9.388112623244524
bw_layer_out:  8
th_layer_out:  5.0812517987440815
--------------------------------------------------------------------------------
Processing layer 65 of 139
Layer Name:res4a_branch2a_relu Type:ReLU
Inputs: ['res4a_branch2a'], Outputs: ['res4a_branch2a']
Min:  0 , Max:  8.695044
n:  128 , len(bin_edges):  635
Mean : th_layer_out:  5.753266206675148 , sf_layer_out:  0.04530130871397754
bw_layer_out:  8
th_layer_out:  5.753266206675148
--------------------------------------------------------------------------------
Processing layer 66 of 139
Layer Name:res4a_branch2b Type:Convolution
Inputs: ['res4a_branch2a'], Outputs: ['res4a_branch2b']
Quantizing conv input layer ... res4a_branch2b
Threshold in shape= (

Mean : th_layer_out:  4.785893436862091 , sf_layer_out:  0.037684200290252684
bw_layer_out:  8
th_layer_out:  4.785893436862091
--------------------------------------------------------------------------------
Processing layer 82 of 139
Layer Name:res4c_branch2b Type:Convolution
Inputs: ['res4c_branch2a'], Outputs: ['res4c_branch2b']
Quantizing conv input layer ... res4c_branch2b
Threshold in shape= ()
Quantizing conv weights for layer res4c_branch2b...
Threshold params shape= (256,)
Min:  0 , Max:  7.357345
n:  128 , len(bin_edges):  635
Mean : th_layer_out:  5.459985601789192 , sf_layer_out:  0.04299201261251333
Threshold out shape= ()
Min:  0 , Max:  7.357345
n:  128 , len(bin_edges):  635
Mean : th_layer_out:  5.459985601789192 , sf_layer_out:  0.04299201261251333
bw_layer_in:  8
th_layer_in:  4.785893436862091
bw_layer_out:  8
th_layer_out:  5.459985601789192
--------------------------------------------------------------------------------
Processing layer 83 of 139
Layer Name:res4c

Mean : th_layer_out:  5.6533598839672585 , sf_layer_out:  0.0445146447556477
bw_layer_in:  8
th_layer_in:  5.28376181795018
bw_layer_out:  8
th_layer_out:  5.6533598839672585
--------------------------------------------------------------------------------
Processing layer 99 of 139
Layer Name:res4e_branch2b_relu Type:ReLU
Inputs: ['res4e_branch2b'], Outputs: ['res4e_branch2b']
Min:  0 , Max:  9.860331
n:  128 , len(bin_edges):  635
Mean : th_layer_out:  5.404518733641327 , sf_layer_out:  0.04255526561922304
bw_layer_out:  8
th_layer_out:  5.404518733641327
--------------------------------------------------------------------------------
Processing layer 100 of 139
Layer Name:res4e_branch2c Type:Convolution
Inputs: ['res4e_branch2b'], Outputs: ['res4e_branch2c']
Quantizing conv input layer ... res4e_branch2c
Threshold in shape= ()
Quantizing conv weights for layer res4e_branch2c...
Threshold params shape= (1024,)
Min:  0 , Max:  8.87228
n:  128 , len(bin_edges):  1269
Mean : th_layer_out

Min:  0 , Max:  9.407513
n:  128 , len(bin_edges):  450
Mean : th_layer_out:  6.6104014381799505 , sf_layer_out:  0.05205040502503898
Threshold out shape= ()
Min:  0 , Max:  9.407513
n:  128 , len(bin_edges):  450
Mean : th_layer_out:  6.6104014381799505 , sf_layer_out:  0.05205040502503898
bw_layer_in:  8
th_layer_in:  5.544451696532113
bw_layer_out:  8
th_layer_out:  6.6104014381799505
--------------------------------------------------------------------------------
Processing layer 116 of 139
Layer Name:res5a_branch2b_relu Type:ReLU
Inputs: ['res5a_branch2b'], Outputs: ['res5a_branch2b']
Min:  0 , Max:  6.7686915
n:  128 , len(bin_edges):  449
Mean : th_layer_out:  4.706355836242437 , sf_layer_out:  0.03705791997041289
bw_layer_out:  8
th_layer_out:  4.706355836242437
--------------------------------------------------------------------------------
Processing layer 117 of 139
Layer Name:res5a_branch2c Type:Convolution
Inputs: ['res5a_branch2b'], Outputs: ['res5a_branch2c']
Quantizing 

Mean : th_layer_out:  4.067860992359264 , sf_layer_out:  0.03203040151463987
bw_layer_out:  8
th_layer_out:  4.067860992359264
--------------------------------------------------------------------------------
Processing layer 133 of 139
Layer Name:res5c_branch2c Type:Convolution
Inputs: ['res5c_branch2b'], Outputs: ['res5c_branch2c']
Quantizing conv input layer ... res5c_branch2c
Threshold in shape= ()
Quantizing conv weights for layer res5c_branch2c...
Threshold params shape= (2048,)
Min:  0 , Max:  24.88694
n:  128 , len(bin_edges):  898
Mean : th_layer_out:  12.166023624381891 , sf_layer_out:  0.09579546160930623
Threshold out shape= ()
Min:  0 , Max:  24.88694
n:  128 , len(bin_edges):  898
Mean : th_layer_out:  12.166023624381891 , sf_layer_out:  0.09579546160930623
bw_layer_in:  8
th_layer_in:  4.067860992359264
bw_layer_out:  8
th_layer_out:  12.166023624381891
--------------------------------------------------------------------------------
Processing layer 134 of 139
Layer Name:

Well done! That concludes the quantizer overview. Now you are ready to put it together and deploy a network/model. 

## [Next: Putting it all together: Compile, Quantize and Deploy][]

[Next: Putting it all together: Compile, Quantize and Deploy]: image_classification_caffe.ipynb